In [1]:
%%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import warnings
import sys
import gc
warnings.filterwarnings('ignore')
print(os.getcwd())
# ../input/feat5-group/test_script_201227
code_path = '/kaggle/input/feat5-group/test_script/test_script/'
sys.path.append(code_path)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   dtype={'row_id': 'int32',
                          'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id': 'int16',
                          'user_answer': 'int8',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )
train = train.loc[train.content_type_id == 0].reset_index(drop=True)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/working
/kaggle/input/feat5-group/tag_w2v_features.csv
/kaggle/input/feat5-group/content_id_real_time.csv
/kaggle/input/feat5-group/model_fusion_800_1000wdata.pkl
/kaggle/input/feat5-group/df_w2v.csv
/kaggle/input/feat5-group/model_new_pipeline_800_1000wdata.pkl
/kaggle/input/feat5-group/tomoyo_cbt_model_0.80247.txt
/kaggle/input/feat5-group/model_new_feat_1000wdata.pkl
/kaggle/input/feat5-group/model_new_pipeline_dropLong_3000wdata.pkl
/kaggle/input/feat5-group/content_id_task_set_distance.csv
/kaggle/input/feat5-group/feat5_gp_v3.npy
/kaggle/input/feat5-group/model_lgb_803.pkl
/kaggle/input/feat5-group/df_svd.csv
/kaggle/input/feat5-group/model_new_pipeline_798_1000wdata.pkl
/kaggle/input/feat5-group/test_script/test_script/test_feat5_group_feature.py
/kaggle/input/feat5-group/test_script/test_script/test_content_encoding.py
/kaggle/input/feat5-group/test_script/test_script/test_global_group_feature.py
/kaggle/input/feat5-group/test_script/test_script/test_part_feature.py
/ka

In [2]:
%%time
# sys.path.append(code_path + 'test_script')
from test_user_feature import initial_dict
from test_state_feature import initial_attempt, dict_trans
from test_content_encoding import content_emb_dict_init_
from test_part_feature import part_dict_init
from test_bundle_feature import bundle_dict_init

user_dict, item_dict = initial_dict(train)
state_dict = initial_attempt(train)
state_dict = dict_trans(state_dict)
path1 = '/kaggle/input/feat5-group/'
path2 = '/kaggle/input/riiid-test-answer-prediction/questions.csv'
content_emb_dict, content_emb_cols = content_emb_dict_init_(train, 
                                                            path = path1, question_path = path2)

questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
map_part_dict = questions.set_index(['question_id'])['part'].to_dict()
train['part'] = train['content_id'].map(map_part_dict)
map_bundle_dict = questions.set_index(['question_id'])['bundle_id'].to_dict()
train['bundle'] = train['content_id'].map(map_bundle_dict)
bundle_dict = bundle_dict_init(train)
# part_dict = part_dict_init(train)

bundle_name = ['bundle_sum', 'bundle_count', 'bundle_mean']
# part_name = ['part_sum', 'part_count', 'part_mean']

prior_question_elapsed_time_mean = train.prior_question_elapsed_time.dropna().values.mean()

initial_dict function took 8.470 s
initial_attempt function took 85.160 s
answer_dict_init_ function took 7.290 s
content_encoder1 function took 39.340 s
content_encoder2 function took 15.440 s
read_enc_data function took 0.290 s
content_emb_dict_init_ function took 62.360 s
bundle_dict_init function took 92.580 s
CPU times: user 3min 44s, sys: 31 s, total: 4min 15s
Wall time: 4min 17s


In [3]:
def get_answer_dict(train):
    item_answer = train.groupby(['content_id', 'user_answer'])['user_answer'].agg('count').unstack()
    item_answer.fillna(0, inplace = True)
    item_answer = item_answer.to_numpy()
    total_sum = np.sum(item_answer, axis = 1)
    for i in range(4):
        item_answer[:, i] = item_answer[:, i]/total_sum
    return item_answer

item_answer_dict = get_answer_dict(train)
del train; gc.collect()

28

In [4]:
import numpy as np
rolling_gp = np.load('/kaggle/input/feat5-group/feat5_gp_v3.npy',allow_pickle=True).item()
whole_group_roll = np.load('../input/riiid-features/whole_group_data_v4_float32.npy', allow_pickle = True).item()

# whole_group_roll = roll_init(whole_group_roll)

In [5]:
from test_user_feature import test_user
from test_state_feature import state_feature
from test_item_feature import test_item
from test_user_feature import update_user_dict
from test_content_encoding import initial_embedding_feature
from test_feat5_group_feature import feat5_group, test_feats5_wrap
from test_rolling_feature import test_rolling_feature_wrapper
from test_part_feature import test_part_feature
from test_bundle_feature import test_bundle_feature
from test_global_group_feature import rolling_feat_group,test_last_group_feature, get_last_name

######
from test_part_feature import update_part_dict
from test_bundle_feature import update_bundle_dict
from test_feat5_group_feature import rolling_df_update

In [6]:
import riiideducation
import pandas as pd

# You can only call make_env() once, so don't lose it!
env = riiideducation.make_env()
# so be careful not to lose it once you start iterating.
iter_test = env.iter_test()
# (test_org, sample_prediction_df) = next(iter_test)


In [7]:
features = ['content_id', 'prior_question_elapsed_time', 'prior_question_had_explanation', 'correct_answer', 
            'user_count', 'user_sum', 'user_mean', 'attempt', 'item_count', 
            'item_sum', 'item_mean', 'answer_ratio_0', 'answer_ratio_1', 'answer_ratio_2', 'answer_ratio_3', 
            'bundle_id', 'part', 'le_tag', 'question_correct_user_ablility_mean',
            'question_correct_user_ablility_median', 'question_wrong_user_ablility_mean', 
            'question_wrong_user_ablility_median', 'word2vec_0', 'word2vec_1', 'word2vec_2', 'word2vec_3',
            'word2vec_4', 'svd_0', 'svd_1', 'svd_2', 'svd_3', 'svd_4', 'tags_w2v_correct_mean_0', 
            'tags_w2v_wrong_mean_0', 'tags_w2v_correct_mean_1', 'tags_w2v_wrong_mean_1', 
            'tags_w2v_correct_mean_2', 'tags_w2v_wrong_mean_2', 'tags_w2v_correct_mean_3', 
            'tags_w2v_wrong_mean_3', 'tags_w2v_correct_mean_4', 'tags_w2v_wrong_mean_4', 
            'real_time_wrong_mean', 'real_time_wrong_median', 'real_time_correct_mean',
            'real_time_correct_median', 'task_set_distance_wrong_mean', 'task_set_distance_wrong_median',
            'task_set_distance_correct_mean', 'task_set_distance_correct_median', 
            'mean_0_ratio', 'mean_1_ratio', 'mean_2_ratio', 'mean_3_ratio', 'mean_4_ratio', 'mean_5_ratio',
            'mean_6_ratio', 'mean_7_ratio', 'mean_8_ratio', 'mean_9_ratio', 'mean_10_ratio', 
            'user_d1', 'user_d2', 'task_set_distance', 'content_shift1', 'item_mean_shift1', 
            'item_mean_diff1', 'user_diff_mean', 'user_diff_std', 'user_diff_min', 'user_diff_max',
            'task_set_item_mean', 'task_set_item_std', 'task_set_item_min', 'task_set_item_max', 
            'task_set_distance2', 'task_distance_shift', 'task_set_distance_diff', 'task_distance_diff_shift',
            'container_mean_1', 'container_std_1', 'container_mean_5', 'container_std_5', 'container_mean_10', 
            'container_std_10', 'container_mean_20', 'container_std_20', 'container_mean_30', 'container_std_30', 
            'container_mean_40', 'container_std_40', 'prior_question_elapsed_time_mean_1', 
            'prior_question_elapsed_time_mean_5', 'prior_question_elapsed_time_mean_10',
            'prior_question_elapsed_time_mean_20', 'prior_question_elapsed_time_mean_30', 
            'prior_question_elapsed_time_mean_40', 'item_mean_mean_1', 'item_mean_mean_5', 'item_mean_mean_10',
            'item_mean_mean_20', 'item_mean_mean_30', 'item_mean_mean_40', 'task_set_distance_mean_1',
            'task_set_distance_mean_5', 'task_set_distance_mean_10', 'task_set_distance_mean_20', 
            'task_set_distance_mean_30', 'task_set_distance_mean_40', 'bundle_sum', 'bundle_count', 
            'bundle_mean', 'begin_time_diff', 'end_time_diff', 'part_duration_time', 'part_time_diff_mean', 
            'part_session_mean', 'part_session_sum', 'part_session_count', 'last_part_item_mean', 
            'last_part_item_median', 'full_group0_item_mean_mean', 'full_group0_item_mean_median', 
            'full_group0_task_set_distance_mean', 'full_group0_task_set_distance_median', 
            'full_group0_timestamp_mean', 'full_group0_timestamp_median', 'full_group1_item_mean_mean', 
            'full_group1_item_mean_median', 'full_group1_task_set_distance_mean',
            'full_group1_task_set_distance_median', 'full_group1_timestamp_mean', 
            'full_group1_timestamp_median', 'part_sum', 'part_count', 'part_mean', 'part_count_global_ratio', 
            'part_sum_global_ratio', 'part_sum_1', 'part_mean_1', 'part_sum_5', 'part_mean_5', 'part_sum_10', 
            'part_mean_10', 'cum_answer0_mean_item_mean', 'cum_answer0_median_item_mean',
            'cum_answer0_mean_task_set_distance', 'cum_answer0_median_task_set_distance', 
            'cum_answer1_mean_item_mean', 'cum_answer1_median_item_mean', 'cum_answer1_mean_task_set_distance', 
            'cum_answer1_median_task_set_distance', 'cum_answer0_time_diff', 'cum_answer1_time_diff', 
            'global_task_set_shift1', 'global_task_set_shift2', 'global_task_set_shift3', 
            'global_task_set_shift4', 'global_task_set_shift5', 'cum_answer0_mean_wrong_time_diff',
            'cum_answer0_median_wrong_time_diff', 'cum_answer1_mean_right_time_diff',
            'cum_answer1_median_right_time_diff', 'last_content_id_time_diff', 'hard_answer0_time', 
            'hard_answer1_time', 'user_trend_mean', 'user_trend_median', 'user_trend_roll_user_ans_sum',
            'user_trend_roll_user_ans_mean', 'user_trend_roll_user_ans_count', 'user_trend_roll_item_ans_sum',
            'user_trend_roll_item_ans_mean', 'user_trend_roll_item_ans_count']

In [8]:
import catboost as cb
model = cb.CatBoostClassifier()

model.load_model('../input/riiid-features/catboost_80588.bin')

# import pickle
# with open('../input/feat5-group/model_new_pipeline_800_1000wdata.pkl', 'rb') as fin:
#     model = pickle.load(fin)

previous_test_df = None
# pbar = tqdm(total=valid.shape[0])
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
ans_map_dict = questions.set_index(['question_id'])['correct_answer'].to_dict()
roll_keep = ['user_id', 'timestamp', 'content_id', 'answered_correctly',
             'prior_question_elapsed_time', 'item_mean', 'task_set_distance', 'part', 'bundle_id', 
             'user_answer', 'correct_answer']

# test_list = []
for (current_test, current_prediction_df) in iter_test:
    if previous_test_df is not None:
        tmp_array = np.array(eval(current_test["prior_group_answers_correct"].iloc[0]))
        tmp_array = tmp_array[tmp_array != -1]
        tmp_ans = np.array(eval(current_test["prior_group_responses"].iloc[0]))
        tmp_ans = tmp_ans[tmp_ans != -1]
        
        previous_test_df['answered_correctly'] = tmp_array
        previous_test_df['user_answer'] = tmp_ans
        user_dict = update_user_dict(user_dict, previous_test_df)
        
#         part_dict = update_part_dict(part_dict, previous_test_df)
        bundle_dict = update_bundle_dict(bundle_dict, previous_test_df)
        

        test_group, test_idx = feat5_group(previous_test_df)
        rolling_gp = rolling_df_update(rolling_gp, test_group)
        
        test_roll_gp, test_roll_idx, name_dict = rolling_feat_group(previous_test_df, roll_keep)
        whole_group_roll = rolling_df_update(whole_group_roll, test_roll_gp)
    
    current_test = current_test[current_test.content_type_id == 0].reset_index(drop = True)

    current_test['answered_correctly'] = np.nan
    current_test['user_answer'] = np.nan
    
    test_ds1 = test_user(current_test, user_dict)
    test_ds2,state_dict = state_feature(current_test, state_dict)
    test_ds3 = test_item(current_test, item_dict)
    current_test[test_ds1.columns] = test_ds1
    current_test[test_ds2.columns] = test_ds2
    current_test[test_ds3.columns] = test_ds3
    
    test_content_emb_ds = initial_embedding_feature(current_test, content_emb_dict)
    current_test[content_emb_cols] = pd.DataFrame(test_content_emb_ds, index = current_test.index)
    
    test_ds5, col_name = test_feats5_wrap(current_test, rolling_gp)
    current_test[col_name] = pd.DataFrame(test_ds5, index = current_test.index)
    
    test_roll_ds, rolling_name = test_rolling_feature_wrapper(current_test, whole_group_roll, roll_keep)
    current_test[rolling_name] = pd.DataFrame(test_roll_ds, index = current_test.index)

    current_test['part'] = current_test['content_id'].map(map_part_dict)
#     test_part_ds = test_part_feature(current_test, part_dict)
#     current_test[part_name] = pd.DataFrame(test_part_ds, index = current_test.index)
    
    current_test['bundle'] = current_test['content_id'].map(map_bundle_dict)
    test_bundle_ds = test_bundle_feature(current_test, bundle_dict)
    current_test[bundle_name] = pd.DataFrame(test_bundle_ds, index = current_test.index)
    
    
    test_gp, valid_idx, name_dict = rolling_feat_group(current_test, roll_keep)
    test_last_group_ds = test_last_group_feature(test_gp, whole_group_roll, name_dict, 
                                                 valid_idx, item_answer_dict)
    last_group_name = get_last_name()
    current_test[last_group_name] = pd.DataFrame(test_last_group_ds, index=current_test.index)
    
    current_test['prior_question_elapsed_time'] = current_test.prior_question_elapsed_time.fillna(prior_question_elapsed_time_mean)
    current_test['prior_question_had_explanation'] = current_test.prior_question_had_explanation.fillna(False).astype('int8')

    current_test['answered_correctly'] = model.predict_proba(current_test[features])[:,1]
#     features = model.feature_name()
#     current_test['answered_correctly'] = model.predict(current_test[features])
#     test_list.append(np.array(current_test['answered_correctly']))
    env.predict(current_test.loc[current_test['content_type_id'] == 0, ['row_id', 'answered_correctly']])
    previous_test_df = current_test.copy()
#     break